# Introduction aux arbres de décision

Comme étudié dans la feuille de TD2, les arbres de décisions structurent un ensemble de règles de décisions sous la forme d'un arbre. Dans ce TP, nous allons utiliser des algorithmes d'apprentissage d'arbres de décision sur des données réelles et étudier leur capacité à prédire la classe de nouvelles données. Cette étude nous permettra d'appréhender le concept d'**over-fitting**.

## A. Préparation du dataset

Le dataset `breast-cancer.data` comporte la description de près de 300 patientes atteintes d'un cancer du sein et examinées dans un institut d'oncologie en Slovénie. La tâche consiste à prédire le risque de récidive pour la patiente à partir des 9 descripteurs catégoriels suivants :

|  Type  |    Catégorie    |
| :---------:|---------|
| age | âge de la patiente|
| menopause | statut ménopausal |
| tumor-size | taille de la tumeur|
| inv-nodes | nombre de ganglions invasifs |
| node-caps | capsule ganglionnaire |
| deg-malig | Degré de malignité |
| breast | sein (gauche ou droit) |
| breast-quad | quadrant mammaire |
| irradiat | radiothérapie |

Chargez le dataset dans un dataframe `patient` et vérifiez les données récupérées (taille, valeurs manquantes, nature des descripteurs, déséquilibre des classes, etc.)

Procédez à l'extraction des descriptions et des labels de classes dans deux tableaux Numpy :
- `X` pour la matrice des descriptions (286 x 9)
- `y` pour le vecteur des labels de classes

## B. One-hot encoding

Utilisez le [`OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) de Scikit-learn pour transformer votre matrice `X` en une matrice binaire `Xbin` où une variable catégorielle initiale avec $k$ modalités sera décomposée en $k$ variables binaires.

In [ ]:
from sklearn.preprocessing import OneHotEncoder


Observez la forme de la matrice récupérée (nombre de variables) et le type particulier de cette matrice. Documentez-vous. 

Vérfiez la transformation opérée sur la première variable catégorielle `age` :
- observez le nombre $k$ de modalités (valeurs) pour cette variable
- vérifiez que chaque ligne de `Xbin` contient exactement une composante à 1 (et toutes les autres à 0) sur les $k$ premières composantes

## C. Apprentissage d'un arbre de décision

Utilisez la librairie Scikit-learn pour découper `Xbin` (et `y`) en deux sous-ensembles (`X_train`, `y_train`) et (`X_test`, `y_test`). On procèdera pour ce TP à un découpage en sous-ensembles de mêmes tailles (50/50).

> <br/>Scikit-learn propose une implémentation assez générique des arbres de décision. Le constructeur d'un [`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier) permet de paramétrer la méthode d'apprentissage ainsi que certaines propriétés attendues sur la structure de l'arbre. Notamment :
> - le paramètre `criterion` definit la mesure de qualité utilisée pour le découpage : `'gini'` (par défaut), `'entropy'` (vu en TD) ou `'log_loss'`
> - le paramètre `min_sample_leaf` definit le nombre minimum d'individus autorisés dans les feuilles de l'arbre (1 par défaut) <br/><br/>
>
> Un arbre de décision (`DecisionTreeClassifier`) peut ensuite être manipulé à partir de deux méthodes :
> - `.fit(X, y)` qui permet d'apprendre un arbre de décision à partir d'un dataset d'entraînement `(X, y)` où <br/> <br/>
>   - `X` contient des descriptions (numériques ou binaires)
>   - `y` contient les labels de classes associés.<br/> <br/>
> - `.predict(X)` qui permet de prédire la classe de chaque individu du dataset `X` <br/>  <br/>

Apprendre un arbre de décision sur les données d'entraînement en utilisant l'entropie comme critère de découpage et en conservant les propriétés structurelles par défaut. Stockez votre modèle de prédiction (arbre de décision) dans une variable `dtree`. 

In [ ]:
from sklearn import tree


La commande suivante vous offre une visualisation du modèle appris.

In [ ]:
plt.figure(figsize=(15,15))
tree.plot_tree(dtree, fontsize=6)
plt.show()

Utilisez les paramètres de votre `OneHotEncoder` pour donner du sens aux descripteurs utilisés dans les noeuds de l'arbre de décision :
- l'attribut `categories_` décrit les nouvelles variables binaires construites à partir de chaque variables catégorielle initiale
- la méthode `.get_feature_names_out()` retourne la liste de toutes les variables binaires

Observez quelques propriétés structurelles de l'arbre de décision :
- nombre de feuilles (sont-elles toutes pures?)
- profondeur de l'arbre 

Vérifiez ces propriétés en utilisant les attributs de votre objet `dtree` (`.get_n_leaves()`, `.get_depth()`)

## D. Over-fitting

Calculez et comparer les taux d'erreurs de prédiction sur :
- les données d'entraînement `X_train` qui ont servi à construire l'arbre de décision
- les données de test `X_test` qui n'ont jamais été utilisées

Comment expliquez-vous cette différence?

Apprenez un nouvel arbre de décision moins profond à partir de `X_train` toujours (par exemple avec `min_sample_leaf=5`). Observez ses propriétés structurelles puis ses performances (taux d'erreur sur `X_train` puis sur `X_test`).

> On parle d'<font color='red'>over-fitting</font> (sur-adéquation aux données) lorsqu'un modèle $f$ appris est très performant sur les données utilisées pour l'apprentissage mais n'ai pas suffisamment générique pour prédire sur de nouvelles données (données de test).
> Pour détecter l'over-fitting on compare :
> - l'<font color='red'>erreur d'apprentissage</font> $e_{train}(f)$: taux d'erreur sur les données d'entraînement (`X_train`)
> - l'<font color='red'>erreur de généralisation</font> $e_{test}(f)$: taux d'erreur sur les données de test (`X_test`)
>
> Le modèle $f$ est sujet à over-fitting si $e_{test}(f) \gg e_{train}(f)$
> <br><br>

> Une manière de réduire l'over-fitting des arbres de décision consiste à limiter la taille des règles de décision (et donc la profondeur de l'arbre) :
> - des règles de décision trop longues ont peu de chance d'être satisfaites par de nouveaux exemples (règles trop spécifiques)
> - des règles de décision courtes ont plus de chance d'être satisfaite par de nouveaux exemples (règles plus générales) <br/><br/>

Ecrire un fonction `eval_DT(X_train, y_train, X_test, y_test, min_samples_leaf)` qui retourne l'erreur d'apprentissage et l'erreur de généralisation d'un arbre de décision de profonduer contrôlée par `min_samples_leaf`, appris sur `X_train` et testé sur `X_test`.

In [ ]:
def eval_DT(X_train, y_train, X_test, y_test, min_samples_leaf):
    ...

Etudiez le phénomène d'over-fitting sur les données `breast-cancer`. Pour cela :
- calculez les erreurs d'apprentissage et de généralisation en faisant évoluer la profondeur de l'arbre (ex. `min_samples_leaf` variant de 1 à 50)
- tracez les courbes associées

Ré-executez votre notebook pour voir dans quelle mesure le phénomène d'over-fitting est sensible au (découpage du) dataset. Faites également évoluer la proportion d'exemples utilisés pour apprendre l'arbre de décision.